In [1]:
import sys
sys.path.append('/home/iceberg/notebooks/PyCon_LT_Workshop')

from helpers.utils import get_spark_session, get_yellow_taxi_data, get_dim_data, ROOT_DIR
from pyspark.sql import functions as f
spark = get_spark_session("broadcasting")

yellow_taxi_data = get_yellow_taxi_data(spark=spark)

dim_taxi_zones, dim_rates, dim_payments, dim_vendor = get_dim_data(spark)

24/02/17 09:27:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# enabling broadcasting hinting
from pyspark.sql.functions import broadcast

# What is broadcasting?

<img src="https://henningkropponlinede.files.wordpress.com/2016/12/spark-broadcast.png?w=313">

## Setting autoBroadcastJoinThreshold

.config("spark.sql.autoBroadcastJoinThreshold", X)

parameter enables datasets which are less then specified size to be broadcasted.
Sometimes it results in huge performance boost.

*-1* -> no broadcasting is automatically done. This was done intentionally to demonstrate broadcasting impact.

*Default is 10MB*. In some cases specifying hints on joins you can make spark use broadcast join.

### How it looks in Spark UI
<img src="https://miro.medium.com/max/2812/1*Pzvy3UDoNIP6g1pOZwu1qw.png">

In [6]:
joined = yellow_taxi_data.join(broadcast(dim_vendor), yellow_taxi_data.VendorID == dim_vendor.vendor_id, "full")
joined.count()
# 24/02/17 09:28:45 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for full outer join.

24/02/17 09:28:45 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for full outer join.
24/02/17 09:28:46 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for full outer join.
24/02/17 09:28:46 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for full outer join.


5443362

In [7]:
joined = yellow_taxi_data.join(broadcast(dim_vendor), yellow_taxi_data.VendorID == dim_vendor.vendor_id, "left")
joined.count()

5443362

## Spark SQL hinting

In [8]:
yellow_taxi_data.createOrReplaceTempView("yellow_taxi_data")
dim_vendor.createOrReplaceTempView("dim_vendor")
spark.sql("""
SELECT /*+ BROADCASTJOIN(dim_vendor) */ * FROM yellow_taxi_data join
 dim_vendor on dim_vendor.vendor_id = yellow_taxi_data.VendorID
""").count()

5430963